# Apprendre des classifieurs avec `scikit-learn`

Pour ce cours, aucune compétence en programmation n'est nécessaire, et tout peut se faire avec weka et le script de vectorisation fourni. Cependant, en dehors de ce cours, il est souvent utile de savoir utiliser un langage de script comme Python pour faire de la classification de documents par apprentissage artificiel.

Celà permet par exemple de modifier et d'étendre facilement les prétraitements, et de réaliser ces derniers et l'apprentissage et l'évaluation de classifieurs dans le même environnement. Cela facilite aussi le passage à l'échelle (par exemple pour des calculs lourds à moindre frais vous pouvez vous tourner ves [Google Collab](https://realpython.com/python-virtual-environments-a-primer/), mais attention à la politique d'usage des données) et vous trouverez plus facilement de la documentation et des cours.

## Considérations pratiques

Pour rendre ce document plus agréable à lire, il est présenté sous forme de notebook [Jupyter](https://jupyter.org/), que vous avez probablement déjà rencontré. C'est un format qui a ses avantages (pratique et lisible) et ses inconvénients (difficile à automatiser, mal versionnable, parfois sujet à erreurs). Tout ce qui est fait ici est tout à fait faisable dans un script indépendant et je vous encourage à écrire un tel script plutôt qu'à réutiliser des notebooks.

Pour pouvoir lancer vous-mêmes et modifier ce notebook, il vous faudra installer quelques dépendances. Je recommande de le faire dans un [environnement virtuel](https://realpython.com/python-virtual-environments-a-primer/) afin de ne pas polluer votre installation principale de Python, mais à défaut, vous pouvez lancer l'appel suivant à `pip` avec le flag `--user`.

**NE JAMAIS UTILISER PIP AVEC `sudo`**

Pour installer les dépendances, ouvrez un terminal et entrez

```console
pip install jupyter scikit-learn
```

Puis démarrez jupyter

```console
jupyter notebook demos/classier_with_sklearn.ipynb
```

Celà devrait vous ouvrir une fenêtre de navigateur sur le notebook, qui est alors prêt à être utilisé.

## Récupérer les données

On va travailler avec le petit corpus habituel. Récupérez le sur [la page du cours](https://loicgrobol.github.io/intro-fouille-textes) ou utilisez la commande suivante pour le télécharge dans un dossier `local/corpus` dans le même dossier que le notebook.

In [ ]:
!mkdir -p local && wget -qO- "https://github.com/LoicGrobol/intro-fouille-textes/releases/download/stable/sample-data.tar.gz" | tar xz -C local --strip-components=2 data/expl3/corpus

Listons les fichiers du corpus

In [ ]:
!ls -Rh local/corpus

local/corpus:
culture  société

local/corpus/culture:
doc0.txt   doc12.txt  doc15.txt  doc3.txt  doc6.txt  doc9.txt
doc10.txt  doc13.txt  doc1.txt   doc4.txt  doc7.txt
doc11.txt  doc14.txt  doc2.txt   doc5.txt  doc8.txt

local/corpus/société:
doc0.txt   doc12.txt  doc15.txt  doc3.txt  doc6.txt  doc9.txt
doc10.txt  doc13.txt  doc1.txt   doc4.txt  doc7.txt
doc11.txt  doc14.txt  doc2.txt   doc5.txt  doc8.txt


## Import

Commençons par importer le package [`scikit-learn`](https://scikit-learn.org), un très bon outil pour l'apprentissage artificiel en Python, sur lequel va s'appuyer la suite de cette démo.

Notez que le nom à utiliser pour l'import est bien `sklearn` pas `scikit-learn`. On importe également les sous-modules qui nous seront utiles.

In [ ]:
import sklearn
import sklearn.datasets

On import maintenant les données : `scikit-learn` a pour ça une commande toute prête, [`sklearn.datasets.load_files`](https://scikit-learn.org/stable/modules/generated/sklearn.datasets.load_files.html), qui prend en entrée exactement le même format que le script de vectorisation — un dossier par classe, un fichier texte par document. `shuffle` sert ici à mélanger les documents pour ne pas avoir tous les documents d'une même classe les uns à la suite des autres.

In [ ]:
dataset = sklearn.datasets.load_files("local/corpus", encoding="utf-8", shuffle=True)

L'objet renvoyé est de type [`sklearn.utils.Bunch`](https://scikit-learn.org/stable/modules/generated/sklearn.utils.Bunch.html) et est prévu pour permettre facilement d'accéder aux documents via son attribut `data`. Ici les 100 premiers caractères du premier document :

In [ ]:
dataset.data[0][:100]

"La française recouvre d'abord l'ensemble des pratiques qui se trouvent sur le territoire français. M"

Les classes sont données par `target` :

In [ ]:
dataset.target[:5]

array([0, 1, 0, 0, 1])

Les indices correspondants aux noms dans `target_names`

In [ ]:
dataset.target_names

['culture', 'société']

## Vectorisation

L'étape suivant est de vectoriser notre corpus : transformer chaque document en une liste de nombres comme on l'a vu dans le cours. `scikit-learn` fournit l'utilitaire [`sklearn.feature_extraction.text.CountVectorizer`](https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html), très complet qui permet d'appliquer une segmentation basique et de transformer des chaînes de caractères en sacs de mots.

Commençons par importer le sous-module dans lequel il se trouve

In [ ]:
import sklearn.feature_extraction.text

Puis appliquons-le à nos données

In [ ]:
vectorizer = sklearn.feature_extraction.text.CountVectorizer()
counts = vectorizer.fit_transform(dataset.data)
print(f"Le corpus a {counts.shape[0]} lignes et {counts.shape[1]} colonnes")
print("Et voici un échantillon :")
counts[:6, :15].todense()

Le corpus a 32 lignes et 11415 colonnes
Et voici un échantillon :


matrix([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 1, 0, 2, 1, 0, 0, 0, 2, 1, 1, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0],
        [0, 1, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]])

Le résultat est une matrice, stockée sous forme creuse pour économiser de la mémoire (on ne stocke que les valeurs non-nulles), d'où l'appel à `todense` pour l'afficher, en interne elle est représentée par une liste de triplets $(\text{ligne}, \text{colonne}, \text{valeur})$ où on ne représente que les valeurs non-nulles

In [ ]:
print(counts[:6, :15])

  (1, 5)	2
  (1, 6)	1
  (1, 11)	1
  (1, 12)	1
  (1, 3)	1
  (1, 10)	2
  (4, 8)	1
  (5, 1)	1
  (5, 4)	2


`CountVectorizer` a de nombreuses options de prétraitement, et je vous encourage à aller lire [sa documentation](https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html) et à expérimenter avec les paramètres. Par exemple pour filtrer les hapax, on peut fixer `min_df` *minimum document frequency* à 2 :

In [ ]:
vectorizer = sklearn.feature_extraction.text.CountVectorizer(min_df=2)
counts = vectorizer.fit_transform(dataset.data)
print(f"Le corpus a {counts.shape[0]} lignes et {counts.shape[1]} colonnes")
print("Et voici un échantillon :")
counts[:6, :15].todense()
 

Le corpus a 32 lignes et 4404 colonnes
Et voici un échantillon :


matrix([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0],
        [0, 0, 4, 0, 0, 0, 3, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [1, 2, 1, 0, 1, 1, 0, 1, 0, 1, 0, 0, 4, 0, 0]])

Notons enfin qu'on utilise ici la fonction `fit_transformer` qui combine deux opérations

- `fit`, qui construit le vocabulaire
- `transform`, qui vectorise les données en utilisant ce vocabulaire

On peut aussi les utiliser à part. Par exemple ici on ne construit le vocabulaire que sur les 10 premiers textes, mais on vectorise tout :

In [ ]:
vectorizer = sklearn.feature_extraction.text.CountVectorizer(min_df=2)
vectorizer.fit(dataset.data[:10])
counts_only10 = vectorizer.transform(dataset.data)
print(f"Le corpus sans les hapax et avec vocabulaire réduit a {counts_only10.shape[0]} lignes et {counts_only10.shape[1]} colonnes") 

Le corpus sans les hapax et avec vocabulaire réduit a 32 lignes et 1370 colonnes


Quelle que soit la façon dont on le fait, un fois que le vectoriser a été `fit`é, on peut le réappliquer à d'autres données : en particulier pour votre corpus de test.

Pour construire un corpus de test on va se contenter d'utiliser un simple split aléatoire, mais il y a aussi moyen de faire de la validation croisée.

In [ ]:
import sklearn.model_selection

In [ ]:
train_set, test_set, train_target, test_target = sklearn.model_selection.train_test_split(dataset.data, dataset.target, test_size=0.33)
vectorizer = sklearn.feature_extraction.text.CountVectorizer(min_df=2)
train_set_counts = vectorizer.fit_transform(train_set)
test_set_counts = vectorizer.transform(test_set)
test_set_counts.todense()

matrix([[0, 0, 0, ..., 0, 0, 0],
        [3, 0, 1, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 1, 0],
        ...,
        [0, 4, 0, ..., 2, 0, 2],
        [3, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]])

## Entraîner des classifieurs

On rentre maintenant dans le vif du sujet : comment utiliser les données pour évaluer et tester un classifieur.

`scikit-learn` a en natif la plupart des classifieurs qu'on peut avoir envie d'utiliser. Commençons avec Naïve Bayes. La version qu'on a vu en cours est disponible dans [`sklearn.naive_bayes.MultinomialNB`](https://scikit-learn.org/stable/modules/naive_bayes.html#multinomial-naive-bayes)

In [ ]:
import sklearn.naive_bayes

Pour créer et entraîner un classifier ça marche exactement comme pour un vectoriser :

In [ ]:
classifier = sklearn.naive_bayes.MultinomialNB()
classifier.fit(train_set_counts, train_target)

MultinomialNB()

On peut alors utiliser le classifieur pour classer de nouvelles données

In [ ]:
classifier.predict(test_set_counts[0])

array([0])

On peut retrouver le nom de sa classe prédite et la comparer à sa classe gold

In [ ]:
predicted_class = dataset.target_names[classifier.predict(test_set_counts[0]).item()]
gold_class = dataset.target_names[test_target[0].item()]
print(f"Classe réelle: {gold_class}, classe prédite {predicted_class}")

Classe réelle: société, classe prédite culture


Si on fait ça sur tout le jeu de test, on récupère l'exactitude du classifieur (en important d'abord `numpy` pour manipuler les résultats).

In [ ]:
import numpy as np

In [ ]:
test_predicted = classifier.predict(test_set_counts)
np.mean(test_predicted == test_target)

0.45454545454545453

Pour une évaluation plus détaillée, `scikit-learn` fournit directement les métriques de classification

In [ ]:
import sklearn.metrics

In [ ]:
print(sklearn.metrics.classification_report(test_target, test_predicted, target_names=dataset.target_names))

              precision    recall  f1-score   support

     culture       0.33      1.00      0.50         3
     société       1.00      0.25      0.40         8

    accuracy                           0.45        11
   macro avg       0.67      0.62      0.45        11
weighted avg       0.82      0.45      0.43        11



## Appliquer des transformations

On peut également appliquer des transformations aux données, par exemple tf⋅idf, qu'on a vu en cours est disponible comme [`sklearn.feature_extraction.text.TfidfTransformer`](https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfTransformer.html), qui est à appliquer après la vectorisation et s'utilise de façon similaire :

In [ ]:
tfidf_transformer = sklearn.feature_extraction.text.TfidfTransformer()
train_set_tfidf = tfidf_transformer.fit_transform(train_set_counts)
test_set_tfidf = tfidf_transformer.transform(test_set_counts)

On récupère alors des représentations qu'on peux apprendre à l'aide d'un classifieur, comme précédemment :

In [ ]:
classifier_tfidf = sklearn.naive_bayes.MultinomialNB()
classifier_tfidf.fit(train_set_tfidf, train_target)
test_tfidf_predicted = classifier.predict(test_set_tfidf)
print(sklearn.metrics.classification_report(test_target, test_tfidf_predicted, target_names=dataset.target_names))

              precision    recall  f1-score   support

     culture       0.33      1.00      0.50         3
     société       1.00      0.25      0.40         8

    accuracy                           0.45        11
   macro avg       0.67      0.62      0.45        11
weighted avg       0.82      0.45      0.43        11



## Autres classifieurs du cours

In [ ]:
import sklearn.neighbors, sklearn.tree, sklearn.svm

Pour nous faciliter la vie : une fonction qui apprend et teste un classifieur :

In [ ]:
def train_eval(classifier, train_set, train_target, test_set, test_target, target_names):
    classifier.fit(train_set, train_target)
    test_predicted = classifier.predict(test_set)
    print(sklearn.metrics.classification_report(test_target, test_predicted, target_names=target_names))

Qu'on va appliquer à tous les classifieurs vus en cours

In [ ]:
for classifier in (
    sklearn.neighbors.KNeighborsClassifier(n_neighbors=3),
    sklearn.tree.DecisionTreeClassifier(),
    sklearn.naive_bayes.MultinomialNB(),
    sklearn.svm.LinearSVC(),
):
    print(type(classifier).__name__)
    train_eval(
        classifier,
        train_set_counts,
        train_target,
        test_set_counts,
        test_target,
        dataset.target_names,
    )


KNeighborsClassifier
              precision    recall  f1-score   support

     culture       0.20      0.67      0.31         3
     société       0.00      0.00      0.00         8

    accuracy                           0.18        11
   macro avg       0.10      0.33      0.15        11
weighted avg       0.05      0.18      0.08        11

DecisionTreeClassifier
              precision    recall  f1-score   support

     culture       0.43      1.00      0.60         3
     société       1.00      0.50      0.67         8

    accuracy                           0.64        11
   macro avg       0.71      0.75      0.63        11
weighted avg       0.84      0.64      0.65        11

MultinomialNB
              precision    recall  f1-score   support

     culture       0.33      1.00      0.50         3
     société       1.00      0.25      0.40         8

    accuracy                           0.45        11
   macro avg       0.67      0.62      0.45        11
weighted avg    